## Importando bibliotecas

In [1]:
import nltk
import pandas as pd

## Definindo função para gerar o vocabulário

In [2]:
#definindo que a função deve receber como parâmetro o caminho de um arquivo de texto
def cria_vocabulario(caminho_texto):
    #utilizando metodo 'open' do python para receber o caminho do arquivo passado e lê-lo
    with open(caminho_texto, "r",encoding="utf8") as f:
        artigo = f.read()
    #utilizando técnicas de NLP para partir o texto em partes menores chamadas de Tokens, que separam palavras de pontos e espaços
    lista_tokens = nltk.tokenize.word_tokenize(artigo)
    lista_palavras = []
    for token in lista_tokens:
        #realizando uma verificação condicional para determinar se o token é alfabético ou não. Se for alfabético será adicionado à lista de palavras.
        if token.isalpha():
            lista_palavras.append(token)
    lista_normalizada = []
    for palavra in lista_palavras:
        #normalizando capa palavra da lista gerada anteriormente para que sejam transferidas para o formato lowecase
        lista_normalizada.append(palavra.lower())
    lista_normalizada
    #retornando uma lista de palavras normalizadas
    return lista_normalizada
#com a função criada, podem ser concatenadas várias listas de geradas a partir de diversos fontes de texto para assim gerar um vocabulário mais extenso e mais preciso.

In [3]:
#concatenando funções de geração de vocabulário para obter um vocabulário mais extenso.
lista_palavras  = cria_vocabulario("dados/txt-dados.txt")
lista_palavras += cria_vocabulario("dados/txt-artigo-dados.txt")
lista_palavras += cria_vocabulario("dados/artigos.txt")

In [4]:
todas_palavras = lista_palavras
#frequencia com que cada palavra da lista de palavras aparece no conjunto
frequencia = nltk.FreqDist(todas_palavras)
#quantidade total de palavras do conjunto
quant_palavras = len(todas_palavras)
#definindo que o vocabulário será a lista de palavras, removendo as aparições repetidas das palavras.
vocabulario = set(lista_palavras)

'''definindo função que calcula a probabilidade da palavra inserida estar correta, dividindo frequencia de surgimento da 
palavra pela quantidade total de palavras.'''
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/quant_palavras

#definindo função que resolve falta de caracteres na palavra digitada
def insere_letras(fatias):
    novas_palavras = []
    letras = "abcdefghijklmnopqrstuvwxyzáéíóúâêôàçñãõ"
    #Recebe duas fatias de uma palavra e insere uma letra entre as fatias
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

#definindo função que resolve caractere incorreto inserido no meio da palavra
def deletando_caracteres(fatias):
    novas_palavras = []
    #para cada letra da palavra, recebe uma duas fatias da palavra e exlui a primeira letra da segunda fatia
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


#criando função que resolve problema de palavra escrita com letras erradas
def troca_letras(fatias):
    novas_palavras = []
    letras = "abcdefghijklmnopqrstuvwxyzáéíóúâêôàçñãõ"
    #remove a primeira letra da segunda fatia e a substitui por outra letra
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

#criando função que soluciona digitação de letras invertidas
def inverte_letras(fatias):
    novas_palavras = []
    #inverte a primeira letra da segunda fatia pela ultima da primeira fatia
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

#Gerando possíveis palavras corretas 
def gerador_palavras(palavra):
    fatias = []
    #criando fatias a partir de palavra inserida
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    #chamando todas as funções anteriores para gerar uma lista com possíveis correções para a palavra inserida
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letras(fatias)
    palavras_geradas += inverte_letras(fatias)
    return palavras_geradas

#definindo o corretor em si
def corretor(palavra):
    #chamando a função de geração de palavras e atribuindo ela à palavras_geradas
    palavras_geradas = gerador_palavras(palavra)
    #definindo que a palavra correta será aquela que tiver a maior porcentagem de aparecimento no nosso conjunto de palavras
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [5]:
#aqui é realizado um pequeno teste para verificar a acurácia das correções através de uma lista de palavras erradas e seu par correto
palavras_teste = pd.read_table("dados/palavras.txt", sep=" ", names = ["correta", "errada"], header=0)
corretas = palavras_teste["correta"]
#definindo palavras erradas
erradas = palavras_teste["errada"]
lista_corrigidas = []
#definindo lista de palavras corrigidas
for errada in erradas:
    lista_corrigidas.append(corretor(errada))
corrigidas = pd.Series(lista_corrigidas)

In [10]:
#definindo função para avaliar acurácia do corretor
def avaliador(correta, corrigida):
    numero_palavras = len(corrigida)
    acertou = 0
    desconhecida = 0
    for i in range(numero_palavras):
        if correta[i] == corrigida[i]:
            acertou += 1
        if correta[i] not in vocabulario:
            desconhecida += 1    
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f"A acurácia do corretor foi de {taxa_acerto}% de {numero_palavras} palavras. Palavras desconhecidas somam {taxa_desconhecida}%")

In [11]:
#chamando função e verificando acurácia do corretor
avaliador(corretas, corrigidas)

A acurácia do corretor foi de 76.22% de 185 palavras, desconhecida é 7.03%


In [ ]:
#vemos então que é possível otimizar o corretor em até 7.03% caso obtenhamos um vocabulário maior